In [2]:
import polars as pl
criteria = pl.read_parquet("inclusion_exclusion_criteria_table.parquet")
criteria_any = pl.read_parquet("inclusion_exclusion_criteria_table_any_lab.parquet")
criteria_one = pl.read_parquet("inclusion_exclusion_criteria_table_one_lab.parquet")

In [20]:
for col in criteria.columns:
    if col == "Global ICU Stay ID":
        continue
    print(f"{col}: {criteria.filter(pl.col(col)==True).height}")

include_adults: 348040
include_severe_acidemia_in_48h_one_lab: 5280
include_severe_acidemia_in_48h_any_lab: 5890
include_sofa_at_any_time: 86055
include_sofa_upto_inclusion: 1575
include_sofa_48h_to_inclusion: 1541
include_lactate_at_any_time: 95221
include_lactate_upto_inclusion: 2862
include_lactate_48h_to_inclusion: 2832
exclude_respiratory_acidosis: 91046
exclude_ketoacidosis: 15156
exclude_prior_RRT_at_any_time: 6200
exclude_prior_RRT_upto_inclusion: 0
exclude_CKD: 13
exclude_gfr_below_30: 67


In [4]:
for col in criteria_any.columns:
    if col == "Global ICU Stay ID":
        continue
    print(f"{col}: {criteria_any.filter(pl.col(col)==True).height}")

include_adults: 348040
include_severe_acidemia_in_48h_one_lab: 5280
include_severe_acidemia_in_48h_any_lab: 5890
include_sofa_at_any_time: 86055
include_sofa_upto_inclusion: 1575
include_sofa_48h_to_inclusion: 1541
include_lactate_at_any_time: 95221
include_lactate_upto_inclusion: 2862
include_lactate_48h_to_inclusion: 2832
exclude_respiratory_acidosis: 91046
exclude_ketoacidosis: 15156
exclude_prior_RRT_at_any_time: 6200
exclude_prior_RRT_upto_inclusion: 0
exclude_CKD: 13
exclude_gfr_below_30: 67
inclusion_time_seconds_any_lab: 0


In [5]:
for col in criteria_one.columns:
    if col == "Global ICU Stay ID":
        continue
    print(f"{col}: {criteria_one.filter(pl.col(col)==True).height}")

include_adults: 348040
include_severe_acidemia_in_48h_one_lab: 5280
include_severe_acidemia_in_48h_any_lab: 5890
include_sofa_at_any_time: 86055
include_sofa_upto_inclusion: 1565
include_sofa_48h_to_inclusion: 1525
include_lactate_at_any_time: 95221
include_lactate_upto_inclusion: 2861
include_lactate_48h_to_inclusion: 2840
exclude_respiratory_acidosis: 91046
exclude_ketoacidosis: 15156
exclude_prior_RRT_at_any_time: 6200
exclude_prior_RRT_upto_inclusion: 0
exclude_CKD: 18
exclude_gfr_below_30: 62
inclusion_time_seconds_one_lab: 0


In [6]:
# Loose inclusion based on labs at any time and acidemia any lab in 48h
loose_inclusion_criteria = ["include_adults", "include_severe_acidemia_in_48h_any_lab", "include_sofa_at_any_time", "include_lactate_at_any_time"]
# Strict inclusion based on labs 48h prior to admission and acidemia in one lab in 48h
# ATTENTION LABS BASED ON INCLUSION TIME BASED ON ANY ACIDEMIA TIME - FIX LATER
strict_inclusion_criteria = ["include_adults", "include_severe_acidemia_in_48h_one_lab", "include_sofa_48h_to_inclusion", "include_lactate_48h_to_inclusion"]
# Exclusion criteria
loose_exclusion_criteria = ["exclude_respiratory_acidosis", "exclude_ketoacidosis", "exclude_prior_RRT_upto_inclusion", "exclude_CKD"]
strict_exclusion_criteria = ["exclude_respiratory_acidosis", "exclude_ketoacidosis", "exclude_prior_RRT_at_any_time", "exclude_CKD"]


In [33]:
def create_flowchart(criteria_table: pl.DataFrame, inclusion_criteria: list[str], exclusion_criteria: list[str]) :
    df = criteria_table

    x = [df.height]

    for col in inclusion_criteria:
        df = df.filter(pl.col(col) == True)
        x.append(col)
        x.append(df.height)
    for col in exclusion_criteria:
        df = df.filter(pl.col(col) == False)
        x.append(col)
        x.append(df.height)
    return x

def print_flowchart(x: list):
    for i, item in enumerate(x): 
        if i % 2 == 0:
            print(f"{item}")
        else:
            print("   |")
            print(f"{item} -----> {x[i-1] - x[i+1]} ")
            print("   |")
            print("   v")

In [34]:
x = create_flowchart(criteria, loose_inclusion_criteria, loose_exclusion_criteria)
print_flowchart(x)

356849
   |
include_adults -----> 8809 
   |
   v
348040
   |
include_severe_acidemia_in_48h_any_lab -----> 342170 
   |
   v
5870
   |
include_sofa_at_any_time -----> 2580 
   |
   v
3290
   |
include_lactate_at_any_time -----> 591 
   |
   v
2699
   |
exclude_respiratory_acidosis -----> 1547 
   |
   v
1152
   |
exclude_ketoacidosis -----> 50 
   |
   v
1102
   |
exclude_prior_RRT_upto_inclusion -----> 0 
   |
   v
1102
   |
exclude_CKD -----> 3 
   |
   v
1099


In [35]:
y = create_flowchart(criteria, strict_inclusion_criteria, strict_exclusion_criteria)
print_flowchart(y)

356849
   |
include_adults -----> 8809 
   |
   v
348040
   |
include_severe_acidemia_in_48h_one_lab -----> 342779 
   |
   v
5261
   |
include_sofa_48h_to_inclusion -----> 3932 
   |
   v
1329
   |
include_lactate_48h_to_inclusion -----> 522 
   |
   v
807
   |
exclude_respiratory_acidosis -----> 369 
   |
   v
438
   |
exclude_ketoacidosis -----> 4 
   |
   v
434
   |
exclude_prior_RRT_at_any_time -----> 21 
   |
   v
413
   |
exclude_CKD -----> 3 
   |
   v
410
